In [0]:
pip install azure-search-documents==11.4.0


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
pip install azure-cosmos==4.9.0

  Obtaining dependency information for azure-cosmos==4.9.0 from https://files.pythonhosted.org/packages/61/dc/380f843744535497acd0b85aacb59565c84fc28bf938c8d6e897a858cd95/azure_cosmos-4.9.0-py3-none-any.whl.metadata
  Using cached azure_cosmos-4.9.0-py3-none-any.whl.metadata (80 kB)
Using cached azure_cosmos-4.9.0-py3-none-any.whl (303 kB)
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%pip install sentence-transformers

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
# Initialize the SearchClient
service_endpoint = "ai-search-endPoint"
admin_key = "admin-key"
index_name = "gold"

In [0]:
# Initialize Cosmos DB client and container
url = "cosmos-db-endpoint"
key = "cosmos-db-key"
database_name = "cosmos-db-name"
silver_container_name = "silver"

In [0]:
from pyspark.sql.types import *
from datetime import datetime
from sentence_transformers import SentenceTransformer
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from azure.cosmos import CosmosClient, PartitionKey
import logging

In [0]:

client = CosmosClient(url, credential=key)
database = client.get_database_client(database_name)
silver_container = database.get_container_client(silver_container_name)

In [0]:
# Retreive the data that are not processed yet
query = "SELECT * FROM c WHERE c.processed = false"
items = silver_container.query_items(query=query, enable_cross_partition_query=True)


In [0]:
items_list =list(items)

In [0]:
len(items_list)


17306

In [0]:
def declare_procced_news(record):
    record["processed"] = True
    return record

In [0]:
if len(items_list) !=0:
    
    new_chunks_df  = spark.createDataFrame(items_list)
    chunks = new_chunks_df.select("chunk_text").rdd.flatMap(lambda x: x).collect()

    # Load a local embedding model
    model = SentenceTransformer('all-MiniLM-L6-v2')

    print("embedding chunks")
    embeddings = model.encode(chunks, batch_size=32, show_progress_bar=True)
    print("embedding is done successfully")



    embeddings_data_list =[]
    for row , embbed_vec  in zip(new_chunks_df.rdd.collect(),embeddings):
        if not row.pub_date:
            continue
        
        data = {
            "id":row.id,
            "embedding_vec": [float(x) for x in list(embbed_vec)],
            "pub_date": datetime.fromisoformat(row.pub_date)

        }
        embeddings_data_list.append(data)

    # # upload documents to Azure Search

    credential = AzureKeyCredential(admin_key)
    search_client = SearchClient(service_endpoint, index_name, credential)


    # Upload the documents
    for embedding_data in embeddings_data_list:
        result = search_client.upload_documents([embedding_data])
        print("vectors embedding are stored successfully")

    # Check if the upload was successful for each document
    for res in result:
        if res.succeeded:
            print(f"Document with ID {res.key} uploaded successfully.")
            break
        else:
            print(f"Failed to upload document with ID {res.key}. Error message: {res.error_message}")



    #update the processed news
    items_list = list(map(declare_procced_news,items_list))
    for item in items_list:
        try:
        
            # Utiliser upsert pour mettre à jour ou insérer le document
            silver_container.upsert_item(body=item)

            logging.info(f"{item['id']} Updated Successefully.")
        except Exception as e:
            logging.info(f"Erreur lors de la mise à jour du document {item['id']}: {str(e)}")
else:
    logging.info("No new news to process")
    print("No new news to process")



embedding chunks


Batches:   0%|          | 0/265 [00:00<?, ?it/s]

embedding is done successfully
vectors embedding are stored successfully
Document with ID 00aa0852-dc2b-4ed4-8d97-d60b05e6c926 uploaded successfully.


In [0]:
# #update the processed news
# # items_list = list(map(declare_procced_news,items_list))
# for item in items_list:
#     try:
#         # Utiliser upsert pour mettre à jour ou insérer le document
#         silver_container.delete_item(body=item,partition_key=item["pub_date"])

#         logging.info(f"Document {item['id']} mis à jour avec succès.")
#     except Exception as e:
#         logging.info(f"Erreur lors de la mise à jour du document {item['id']}: {str(e)}")

In [0]:
# #update the processed news
# # items_list = list(map(declare_procced_news,items_list))
# for item in items_list:
#     try:
#         item["processed"]=False
#         # Utiliser upsert pour mettre à jour ou insérer le document
#         silver_container.upsert_item(body=item)

#         logging.info(f"Document {item['id']} mis à jour avec succès.")
#     except exceptions.CosmosHttpResponseError as e:
#         logging.info(f"Erreur lors de la mise à jour du document {item['id']}: {str(e)}")